In [1]:
# Importações
from pybit.unified_trading import HTTP
import pandas as pd
import numpy as np
from datetime import datetime

# Variáveis globais
cliente = HTTP()

data_inicio = '2025-01-01'
data_fim = datetime.now()
saldo = 1000
cripto = 'BTCUSDT'
time_frame = '15'

# Busca de velas e somar todas as velas entre a data de início e a data de fim
data_inicio_timestamp = int( pd.to_datetime(data_inicio).timestamp() * 1000 )
data_fim_timestamp = int( pd.to_datetime(data_fim).timestamp() * 1000 )

velas_sem_estrutura = []

print('Carregando dados históricos...')
# loop para somar todas as velas entre a data de início e a data de fim
while data_inicio_timestamp < data_fim_timestamp:
    resposta = cliente.get_kline(symbol=cripto, interval=time_frame, limit=1000, start=data_inicio_timestamp)
    velas_sem_estrutura = velas_sem_estrutura + resposta['result']['list'][ : :-1]
    data_inicio_timestamp = int( velas_sem_estrutura[-1][0] ) + 1000

# Criação do DataFrame
colunas = ['tempo_abertura', 'abertura', 'maxima', 'minima', 'fechamento', 'volume', 'turnover']
df = pd.DataFrame(velas_sem_estrutura, columns=colunas)

# TIMESTAMP -> DATETIME
df['tempo_abertura'] = pd.to_datetime(df['tempo_abertura'].astype(np.int64), unit='ms')

# Define tempo_abertura como índice
df.set_index('tempo_abertura', inplace=True)

# Tratamento dos dados para converter de texto para número com vírgula
df['abertura'] = df['abertura'].astype(float)
df['maxima'] = df['maxima'].astype(float)
df['minima'] = df['minima'].astype(float)
df['fechamento'] = df['fechamento'].astype(float)
df['volume'] = df['volume'].astype(float)
df['turnover'] = df['turnover'].astype(float)

df['ema_9'] = df['fechamento'].ewm(span=9, adjust=False).mean()
df['ema_21'] = df['fechamento'].ewm(span=21, adjust=False).mean()

df

Carregando dados históricos...


,abertura,maxima,minima,fechamento,volume,turnover,ema_9,ema_21
tempo_abertura,,,,,,,,
2025-01-01 00:00:00,93530.0,93672.8,93451.0,93623.1,579.184,5.418099e+07,93623.100000,93623.100000
2025-01-01 00:15:00,93623.1,93876.8,93583.6,93720.0,691.964,6.486769e+07,93642.480000,93631.909091
2025-01-01 00:30:00,93720.0,93925.0,93719.9,93838.3,434.499,4.077560e+07,93681.644000,93650.671901
2025-01-01 00:45:00,93838.3,94457.9,93810.0,94343.2,1447.694,1.363153e+08,93813.955200,93713.629001
2025-01-01 01:00:00,94343.2,94360.0,93868.0,94093.8,1121.619,1.054882e+08,93869.924160,93748.190001
...,...,...,...,...,...,...,...,...
2025-05-08 16:45:00,101114.0,101300.0,100997.5,101227.1,1123.111,1.136645e+08,100679.684004,100164.944920
2025-05-08 17:00:00,101227.1,101252.0,101061.2,101202.9,733.673,7.422950e+07,100784.327203,100259.304473
2025-05-08 17:15:00,101202.9,101687.9,101202.9,101475.3,2358.933,2.393632e+08,100922.521763,100369.849521


In [2]:
df_diario = df.resample('D').last()
df_diario['ema_9_1d'] = df_diario['fechamento'].ewm(span=9, adjust=False).mean()
df_diario['ema_21_1d'] = df_diario['fechamento'].ewm(span=21, adjust=False).mean()
df_diario['fechamento_1d'] = df_diario['fechamento']

df_diario


,abertura,maxima,minima,fechamento,volume,turnover,ema_9,ema_21,ema_9_1d,ema_21_1d,fechamento_1d
tempo_abertura,,,,,,,,,,,
2025-01-01,94700.0,94732.3,94549.0,94564.6,399.304,3.778056e+07,94702.917209,94612.606349,94564.600000,94564.600000,94564.6
2025-01-02,96962.4,96962.5,96807.2,96960.5,357.372,3.462512e+07,96969.582815,97030.786900,95043.780000,94782.409091,96960.5
2025-01-03,98125.4,98146.9,98070.4,98140.0,167.137,1.639543e+07,98210.328667,98202.726830,95663.024000,95087.644628,98140.0
2025-01-04,98179.7,98270.1,98160.0,98177.2,153.428,1.506756e+07,98240.444028,98216.740699,96165.859200,95368.513298,98177.2
2025-01-05,98399.9,98464.1,98271.2,98340.5,395.443,3.889197e+07,98424.096560,98309.791439,96600.787360,95638.693908,98340.5
...,...,...,...,...,...,...,...,...,...,...,...
2025-05-04,94107.6,94300.0,94107.6,94224.1,1133.881,1.068203e+08,94694.667963,95042.201774,94525.797805,91744.123556,94224.1
2025-05-05,94787.3,94800.0,94680.0,94698.7,400.491,3.794282e+07,94781.479166,94640.894622,94560.378244,92012.721415,94698.7
2025-05-06,96829.8,96880.6,96726.1,96799.0,1349.710,1.306584e+08,96114.967965,95512.932992,95008.102595,92447.837650,96799.0


In [3]:
# Shift dos valores diários para que cada dia use o valor do dia anterior
# Shift dos valores diários para o dia anterior
df_diario_shifted = df_diario[['ema_9_1d', 'ema_21_1d', 'fechamento_1d']].shift(1)
df_diario_shifted.dropna(inplace=True)
df_diario_shifted

,ema_9_1d,ema_21_1d,fechamento_1d
tempo_abertura,,,
2025-01-02,94564.600000,94564.600000,94564.6
2025-01-03,95043.780000,94782.409091,96960.5
2025-01-04,95663.024000,95087.644628,98140.0
2025-01-05,96165.859200,95368.513298,98177.2
2025-01-06,96600.787360,95638.693908,98340.5
...,...,...,...
2025-05-04,94601.222257,91496.125912,95815.5
2025-05-05,94525.797805,91744.123556,94224.1
2025-05-06,94560.378244,92012.721415,94698.7


In [4]:
# Join com os dados de 15 minutos
df_new = df.join(df_diario_shifted, on='tempo_abertura', how='left')
df_new.tail(60)

# Outra forma de unir os dados
# df_new = pd.merge_asof(
#     df.sort_index(), 
#     df_diario.sort_index(), 
#     left_index=True, 
#     right_index=True,
#     direction='backward'
# )

,abertura,maxima,minima,fechamento,volume,turnover,ema_9,ema_21,ema_9_1d,ema_21_1d,fechamento_1d
tempo_abertura,,,,,,,,,,,
2025-05-08 03:00:00,98636.1,98838.5,98484.8,98731.9,1357.510,1.339362e+08,98239.540039,97748.186016,NaN,NaN,NaN
2025-05-08 03:15:00,98731.9,98800.0,98646.0,98756.8,601.852,5.943438e+07,98342.992031,97839.878196,NaN,NaN,NaN
2025-05-08 03:30:00,98756.8,99099.2,98666.0,99074.9,1840.646,1.820402e+08,98489.373625,97952.152905,NaN,NaN,NaN
2025-05-08 03:45:00,99074.9,99187.9,98800.0,99105.8,2067.295,2.046842e+08,98612.658900,98057.029914,NaN,NaN,NaN
2025-05-08 04:00:00,99105.8,99361.1,99060.0,99169.8,2493.714,2.474085e+08,98724.087120,98158.190831,NaN,NaN,NaN
2025-05-08 04:15:00,99169.8,99393.1,99164.5,99339.9,1596.895,1.585771e+08,98847.249696,98265.618937,NaN,NaN,NaN
2025-05-08 04:30:00,99339.9,99340.0,99099.9,99126.1,1682.149,1.669061e+08,98903.019757,98343.844488,NaN,NaN,NaN
2025-05-08 04:45:00,99126.1,99183.4,98903.1,98918.4,1439.274,1.425177e+08,98906.095805,98396.076808,NaN,NaN,NaN
2025-05-08 05:00:00,98918.4,99098.4,98863.4,98896.4,884.128,8.750405e+07,98904.156644,98441.560734,NaN,NaN,NaN


In [5]:
# Preencher valores NaN com o primeiro valor do dia
df_new['ema_9_1d'] = df_new['ema_9_1d'].ffill()
df_new['ema_21_1d'] = df_new['ema_21_1d'].ffill()
df_new['fechamento_1d'] = df_new['fechamento_1d'].ffill()
df_new.tail(60)

,abertura,maxima,minima,fechamento,volume,turnover,ema_9,ema_21,ema_9_1d,ema_21_1d,fechamento_1d
tempo_abertura,,,,,,,,,,,
2025-05-08 03:00:00,98636.1,98838.5,98484.8,98731.9,1357.510,1.339362e+08,98239.540039,97748.186016,95404.102076,92860.588773,96988.1
2025-05-08 03:15:00,98731.9,98800.0,98646.0,98756.8,601.852,5.943438e+07,98342.992031,97839.878196,95404.102076,92860.588773,96988.1
2025-05-08 03:30:00,98756.8,99099.2,98666.0,99074.9,1840.646,1.820402e+08,98489.373625,97952.152905,95404.102076,92860.588773,96988.1
2025-05-08 03:45:00,99074.9,99187.9,98800.0,99105.8,2067.295,2.046842e+08,98612.658900,98057.029914,95404.102076,92860.588773,96988.1
2025-05-08 04:00:00,99105.8,99361.1,99060.0,99169.8,2493.714,2.474085e+08,98724.087120,98158.190831,95404.102076,92860.588773,96988.1
2025-05-08 04:15:00,99169.8,99393.1,99164.5,99339.9,1596.895,1.585771e+08,98847.249696,98265.618937,95404.102076,92860.588773,96988.1
2025-05-08 04:30:00,99339.9,99340.0,99099.9,99126.1,1682.149,1.669061e+08,98903.019757,98343.844488,95404.102076,92860.588773,96988.1
2025-05-08 04:45:00,99126.1,99183.4,98903.1,98918.4,1439.274,1.425177e+08,98906.095805,98396.076808,95404.102076,92860.588773,96988.1
2025-05-08 05:00:00,98918.4,99098.4,98863.4,98896.4,884.128,8.750405e+07,98904.156644,98441.560734,95404.102076,92860.588773,96988.1
